### Effective Mandays % - West Clients - Souvik

In [597]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [598]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [599]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [600]:
# Python Master for ISPStore
sheet = client.open_by_key("10tpkruv13OLpXYCgiua0Vnn_QEqWWVq9D9JWpXBNTn4") # Master Sheets ( Dimensions Sheet)

In [601]:
Schema_Master = sheet.worksheet("Schema_Master")
Schema_Master = get_as_dataframe(Schema_Master)
Data = sheet.worksheet("Data")

In [602]:
Schema_Master= Schema_Master[Schema_Master['flag']==1]
Schema_Master.head()

Project          SQL Schema Name   From Date     To Date  flag  \
0   Amul GT & MT         [AmulManagement]  01-11-2025  30-11-2025   1.0   
1    AWL Agri-GT  [AdaniWilmerManagement]  01-11-2025  30-11-2025   1.0   
7       Cholayil     [CholayilManagement]  01-11-2025  30-11-2025   1.0   
9      Danone-GT     [DanoneGTManagement]  01-11-2025  30-11-2025   1.0   
10     Danone-MT     [DanoneMTManagement]  01-11-2025  30-11-2025   1.0   

   Region                         Unnamed: 6  \
0    West  =DATE(YEAR(C2),MONTH(C2),DAY(C2))   
1    West                                NaN   
7    West                                NaN   
9    West                                NaN   
10   West                                NaN   

                           Unnamed: 7  
0   =DATE(YEAR(D2),MONTH(D2),DAY(D2))  
1                                 NaN  
7                                 NaN  
9                                 NaN  
10                                NaN

In [603]:
x= Schema_Master['SQL Schema Name'].to_list()
projects = [str(p).strip("[]") for p in Schema_Master['Project'].tolist()]

In [604]:
date_ranges = [
    (
        pd.to_datetime(row["From Date"]).strftime("%Y-%m-%d"),
        pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")
    )
    for _, row in Schema_Master.iterrows()
]


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_25752\1068834426.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")


In [605]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [606]:
conn = pyodbc.connect(conn_str)

In [607]:
final_df = pd.DataFrame()

In [608]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f"""
WITH CTE AS (
    SELECT 
        '{project}' AS SchemaName,
        (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
        TXT.Visit_Date,TXT.UserName,
        (TXT.Visit_Date + ' ' + TXT.CHECKIN_TIME)  AS CheckinTime,
		TXT.CHECKIN_TIME,TXT.LISStoreCode,
        (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
        TXT.StoreName,
        TXT.Designation,
        (TXT.Visit_Date + ' ' + TXT.CHECKOUT_TIME) AS CheckoutTime,
        TXT.CHECKOUT_TIME,
		TXT.FullName,
        TXT.NoPermissionReason,
        1 AS VisitFlag,
        DATEPART(DAY, TRY_CONVERT(date, TXT.Visit_Date, 105)) AS DayNum
    FROM (
        SELECT
            FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
            SA.LISStoreCode,
            SA.StoreName,
            SA.UserName,
            SA.Designation,
            SA.FullName,
            CONVERT(varchar(8), CAST([CheckIn-Time]  AS time), 108) AS CHECKIN_TIME,
            CONVERT(varchar(8), CAST([CheckOut-Time] AS time), 108) AS CHECKOUT_TIME,
            SA.NoPermissionReason
        FROM [PowerBI_LISV2].{value} .[ISPStoreAttendance_NormalExport] AS SA
        LEFT JOIN (
            SELECT K.*
            FROM (SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.{value} .UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS UM ON UM.UserName = SA.UserName
        WHERE 
            -- SA.[VisitDate] BETWEEN '{date_from}' AND '{date_to}'
            -- FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') BETWEEN '{date_from}' AND '{date_to}'
            -- current month year filter
            [Month] = MONTH(EOMONTH(GETDATE()))-1
            AND [Year]  = YEAR(EOMONTH(GETDATE()))
			-- True is considered here for all active users 
            -- AND UM.Status = 'True'
            AND SA.UserName NOT LIKE '%Star%'
            AND LOWER(SA.UserName) NOT LIKE '%test%'
    ) AS TXT
)
SELECT K.*,SUP.Supervisor
FROM
(SELECT cte.SchemaName,cte.Visit_Date,cte.UserName,cte.FullName,cte.Designation,
count(cte.LISStoreCode) as StoreCount,
min(CHECKIN_TIME) as Store_CheckIn,max(CHECKOUT_TIME) as Store_CheckOut 
FROM CTE
-- WHERE cte.Visit_Date = '27-10-2025'
-- WHERE TRY_CONVERT(date, cte.Visit_Date, 105) BETWEEN '{date_from}' AND '{date_to}'
GROUP BY cte.SchemaName,cte.Visit_Date,cte.UserName,cte.FullName,cte.Designation) AS K
LEFT JOIN (SELECT ISP,Supervisor FROM
		(SELECT *,RANK() OVER (PARTITION BY UH.ISP ORDER BY UH.UserHierarchyListId DESC) AS RK FROM
			[PowerBI_LISV2].{value}.UserHierarchyListReport AS UH)AS UH WHERE UH.RK =1) AS SUP
ON SUP.ISP = K.UserName;"""
        df = pd.read_sql(query, conn)
        final_df = pd.concat([final_df, df], ignore_index=True)
        print(f"Running for: {value} {date_from} {date_to}")



Running for: [AmulManagement] 2025-01-11 2025-11-30
Running for: [AdaniWilmerManagement] 2025-01-11 2025-11-30
Running for: [CholayilManagement] 2025-01-11 2025-11-30
Running for: [DanoneGTManagement] 2025-01-11 2025-11-30
Running for: [DanoneMTManagement] 2025-01-11 2025-11-30
Running for: [GCPLManagement] 2025-01-11 2025-11-30
Running for: [GCPLFoodsManagement] 2025-01-11 2025-11-30
Running for: [JohnsonJohnsonManagement] 2025-01-11 2025-11-30
Running for: [ParleManagement] 2025-01-11 2025-11-30
Running for: [PidiliteManagement] 2025-01-11 2025-11-30
Running for: [PiramalManagement] 2025-01-11 2025-11-30
Running for: [ZydusMTManagement] 2025-01-11 2025-11-30
Running for: [ZydusGTManagement] 2025-01-11 2025-11-30


In [609]:
# df_User_final =pd.DataFrame()

In [610]:
# for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
#         query = f"""
#         SELECT K.UserNameId ,K.SupervisorName
#         FROM (
#             SELECT UserName as UserNameId,FullName AS SupervisorName,
#                    RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
#             FROM PowerBI_LISV2.{value} .UserMaster
#             WHERE UserName LIKE '%SUP%'
#         ) AS K
#         WHERE K.RNK = 1;
#         """
#         df_User = pd.read_sql(query, conn)
#         df_User_final = pd.concat([df_User_final, df_User], ignore_index=True)


In [611]:
# df_User_final.head()

In [612]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102171 entries, 0 to 102170
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   SchemaName      102171 non-null  object
 1   Visit_Date      102171 non-null  object
 2   UserName        102171 non-null  object
 3   FullName        102171 non-null  object
 4   Designation     102171 non-null  object
 5   StoreCount      102171 non-null  int64 
 6   Store_CheckIn   102171 non-null  object
 7   Store_CheckOut  102169 non-null  object
 8   Supervisor      97926 non-null   object
dtypes: int64(1), object(8)
memory usage: 7.0+ MB


In [613]:
# final_df = pd.merge(final_df,df_User_final,left_on='Supervisor', right_on='UserNameId', how= 'left')

In [614]:
final_df.head()

SchemaName  Visit_Date  UserName                  FullName   Designation  \
0  Amul GT & MT  01-11-2025  AMLMR027  Tejaswini Sachin Shedage  Merchandiser   
1  Amul GT & MT  02-11-2025  AMLMR027  Tejaswini Sachin Shedage  Merchandiser   
2  Amul GT & MT  04-11-2025  AMLMR027  Tejaswini Sachin Shedage  Merchandiser   
3  Amul GT & MT  05-11-2025  AMLMR027  Tejaswini Sachin Shedage  Merchandiser   
4  Amul GT & MT  06-11-2025  AMLMR027  Tejaswini Sachin Shedage  Merchandiser   

   StoreCount Store_CheckIn Store_CheckOut  Supervisor  
0           3      11:45:50       20:48:32  AML-SUP-11  
1           1      11:09:08       20:21:49  AML-SUP-11  
2           1      10:58:15       20:30:47  AML-SUP-11  
3           2      11:02:49       20:35:05  AML-SUP-11  
4           2      11:05:00       20:24:03  AML-SUP-11

In [615]:
final_df = final_df.drop(columns = ['Supervisor'])

In [616]:
# # Convert to datetime.time (using a dummy date)
# final_df['Store_CheckIn1'] = pd.to_datetime(final_df['Store_CheckIn'], format='%H:%M:%S')
# final_df['Store_CheckOut1'] = pd.to_datetime(final_df['Store_CheckOut'], format='%H:%M:%S')

# # # Convert timedelta to hours
# # final_df['Work_Hours'] = (final_df['Store_CheckOut'] - final_df['Store_CheckIn']).dt.total_seconds() / 3600
# # # Convert duration to HH:MM format
# final_df['FOS'] = (final_df['Store_CheckOut1'] - final_df['Store_CheckIn1']).apply(lambda x: f"{int(x.total_seconds()//3600):02d}:{int((x.total_seconds()%3600)//60):02d}")

# # Create flag for > 6:30 hours
# final_df['FOS_MoreThan_6_30'] = (final_df['Store_CheckOut1'] - final_df['Store_CheckIn1']).apply(lambda x: 1 if x.total_seconds() > (6.5 * 3600) else 0)

In [617]:

# Convert to datetime (dummy date added automatically)
final_df['Store_CheckIn1'] = pd.to_datetime(final_df['Store_CheckIn'], format='%H:%M:%S', errors='coerce')
final_df['Store_CheckOut1'] = pd.to_datetime(final_df['Store_CheckOut'], format='%H:%M:%S', errors='coerce')

# Calculate duration safely (handle NaT)
def calculate_duration(row):
    if pd.notnull(row['Store_CheckIn1']) and pd.notnull(row['Store_CheckOut1']):
        diff = row['Store_CheckOut1'] - row['Store_CheckIn1']
        hours = int(diff.total_seconds() // 3600)
        minutes = int((diff.total_seconds() % 3600) // 60)
        return f"{hours:02d}:{minutes:02d}"
    else:
        return np.nan

final_df['FOS'] = final_df.apply(calculate_duration, axis=1)

# Create flag for > 6:30 hours safely
def flag_more_than_6_30(row):
    if pd.notnull(row['Store_CheckIn1']) and pd.notnull(row['Store_CheckOut1']):
        diff = row['Store_CheckOut1'] - row['Store_CheckIn1']
        return 1 if diff.total_seconds() > (6.5 * 3600) else 0
    else:
        return np.nan

final_df['FOS_MoreThan_6_30'] = final_df.apply(flag_more_than_6_30, axis=1)


In [618]:
final_df.drop(columns=['Store_CheckIn1', 'Store_CheckOut1','FullName'], inplace=True)

In [619]:
final_df.head()

SchemaName  Visit_Date  UserName   Designation  StoreCount Store_CheckIn  \
0  Amul GT & MT  01-11-2025  AMLMR027  Merchandiser           3      11:45:50   
1  Amul GT & MT  02-11-2025  AMLMR027  Merchandiser           1      11:09:08   
2  Amul GT & MT  04-11-2025  AMLMR027  Merchandiser           1      10:58:15   
3  Amul GT & MT  05-11-2025  AMLMR027  Merchandiser           2      11:02:49   
4  Amul GT & MT  06-11-2025  AMLMR027  Merchandiser           2      11:05:00   

  Store_CheckOut    FOS  FOS_MoreThan_6_30  
0       20:48:32  09:02                1.0  
1       20:21:49  09:12                1.0  
2       20:30:47  09:32                1.0  
3       20:35:05  09:32                1.0  
4       20:24:03  09:19                1.0

In [620]:
final_df_attendance = pd.DataFrame()

In [621]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f"""
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        M.FullName,
        M.Gender,
        M.Region,
        M.State,
        M.City,
        M.Designation,
        TRY_CONVERT(date, M.LastWorkingDate, 105)  AS LastWorkingDate,
        M.Status AS EmpStatus,
        -- A.Status AS AttRawStatus,
        -- A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present'                              THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL           THEN 'A'
            WHEN A.Status = 'Leave'                                THEN 'L'
            WHEN A.Status IN ('Compensatory Off', 'Comp Off')      THEN 'CO'
            WHEN A.Status IN ('Week Off', 'WeekOff')               THEN 'WO'
            WHEN A.Status IN ('Meeting', 'Meeting/Training')       THEN 'M'
            WHEN A.Status = 'Training'                             THEN 'T'
            WHEN A.Status IN ('National Holiday', 'National Hoilday', 'Holiday', 'Regional Holiday')  THEN 'H'
            WHEN A.Status IN ('Bandh/Strike', 'Bandh Strike', 'Bandh / Strike', 'Market closed', 'Strike')  THEN 'B'
            WHEN A.Status = 'Vacant'                               THEN 'V'
            WHEN A.Status IN ('Personal', 'Personal Leave')        THEN 'PL'
            WHEN A.Status = 'Casual Leave'                         THEN 'CL'
            WHEN A.Status = 'Half Day'                             THEN 'HD'
            ELSE 'Condition not found'
        END                                                   AS AttStatus,
        FORMAT(CONVERT(datetime, A.[Date], 105), 'dd-MM-yyyy') AS Date2,
        -- TRY_CONVERT(date, A.[Date], 105) AS Date2,
        RANK() OVER (
            PARTITION BY A.UserName, A.[Date]
            ORDER BY A.ISPAttendanceMasterExportId DESC
        )                                                      AS rk
    FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
    LEFT JOIN (
        SELECT K.*
        FROM (
            SELECT *,
                   RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
            FROM PowerBI_LISV2.{value}.UserMaster
        ) AS K
        WHERE K.RNK = 1
    ) AS M
        ON A.UserName = M.UserName
    WHERE
        -- Filter by specific date range if needed:
        -- TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
        [Month] = MONTH(EOMONTH(GETDATE()))-1
        AND [Year] = YEAR(EOMONTH(GETDATE()))
        AND M.UserRole NOT IN (
            'MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
            'Senior Manager','Supervisor','National'
        )
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%'
)
SELECT *
FROM CTE
WHERE rk = 1;
;"""
        df = pd.read_sql(query, conn)
        final_df_attendance = pd.concat([final_df_attendance, df], ignore_index=True)
        print(f"Running for: {value} ")



Running for: [AmulManagement] 
Running for: [AdaniWilmerManagement] 
Running for: [CholayilManagement] 
Running for: [DanoneGTManagement] 
Running for: [DanoneMTManagement] 
Running for: [GCPLManagement] 
Running for: [GCPLFoodsManagement] 
Running for: [JohnsonJohnsonManagement] 
Running for: [ParleManagement] 
Running for: [PidiliteManagement] 
Running for: [PiramalManagement] 
Running for: [ZydusMTManagement] 
Running for: [ZydusGTManagement] 


In [622]:
final_df_attendance.head()

SchemaName  UserName    EmployeeID                  FullName Gender  \
0  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   
1  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   
2  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   
3  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   
4  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   

  Region        State    City   Designation LastWorkingDate EmpStatus  \
0   West  Maharashtra  Mumbai  Merchandiser            None      True   
1   West  Maharashtra  Mumbai  Merchandiser            None      True   
2   West  Maharashtra  Mumbai  Merchandiser            None      True   
3   West  Maharashtra  Mumbai  Merchandiser            None      True   
4   West  Maharashtra  Mumbai  Merchandiser            None      True   

  AttStatus       Date2  rk  
0         P  01-11-2025   1  
1         P  02-11-2025   1  
2        WO  03-11-2025   1  
3         P  04-11-2025   1  
4         P  05-11-2025   1

In [623]:
# UserSupervisorMapping
df_User_final =pd.DataFrame()

for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f"""
        SELECT K.ISP, UM.FullName AS Supervisor
        FROM
        (SELECT DISTINCT ISP,Supervisor FROM PowerBI_LISV2.{value}.UserHierarchyListReport)AS K
        LEFT JOIN
        (SELECT K.UserName,K.FullName
            FROM (SELECT UserName,FullName,RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                    FROM PowerBI_LISV2.{value}.UserMaster
        			WHERE UserName LIKE '%SUP%'
                ) AS K
        WHERE K.RNK = 1) AS UM ON UM.UserName = K.Supervisor;
        """
        df_User = pd.read_sql(query, conn)
        df_User_final = pd.concat([df_User_final, df_User], ignore_index=True)

In [624]:
df_User_final.head()

ISP  Supervisor
0  AMLMR015  Pavithra T
1  AMLMR028  Pavithra T
2  AMLMR040  Pavithra T
3  AMLMR041  Pavithra T
4  AMLMR057  Pavithra T

In [625]:
final_df_attendance = pd.merge(final_df_attendance,df_User_final, left_on = 'UserName', right_on = 'ISP',how='left')

In [626]:
final_emd_df = pd.merge(final_df_attendance[['SchemaName','UserName','FullName','EmpStatus','Date2','AttStatus','Region','Supervisor']],final_df,left_on=['SchemaName','UserName','Date2'], right_on=['SchemaName','UserName','Visit_Date'],how = 'left')

In [627]:
final_emd_df=final_emd_df.fillna('')

In [628]:
# final_emd_df=final_emd_df.drop(columns=['SupervisorName'])

In [629]:
final_emd_df.head()

SchemaName  UserName                  FullName EmpStatus       Date2  \
0  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  01-11-2025   
1  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  02-11-2025   
2  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  03-11-2025   
3  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  04-11-2025   
4  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  05-11-2025   

  AttStatus Region           Supervisor  Visit_Date   Designation StoreCount  \
0         P   West  Mahalaxmi Ajit Pady  01-11-2025  Merchandiser        3.0   
1         P   West  Mahalaxmi Ajit Pady  02-11-2025  Merchandiser        1.0   
2        WO   West  Mahalaxmi Ajit Pady                                        
3         P   West  Mahalaxmi Ajit Pady  04-11-2025  Merchandiser        1.0   
4         P   West  Mahalaxmi Ajit Pady  05-11-2025  Merchandiser        2.0   

  Store_CheckIn Store_CheckOut    FOS FOS_MoreThan_6_30  
0      11:45:50       20:48:32  09:02               1.0  
1      11:09:08       20:21:49  09:12               1.0  
2                                                        
3      10:58:15       20:30:47  09:32               1.0  
4      11:02:49       20:35:05  09:32               1.0

In [630]:
final_emd_df.to_csv('souvikfinal_emd_df.csv')

In [631]:
#Sending df on sheet
Data.batch_clear(["A:P"])
set_with_dataframe(Data, final_emd_df)

In [632]:
final_emd_df.head()

SchemaName  UserName                  FullName EmpStatus       Date2  \
0  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  01-11-2025   
1  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  02-11-2025   
2  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  03-11-2025   
3  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  04-11-2025   
4  Amul GT & MT  AMLMR027  Tejaswini Sachin Shedage      True  05-11-2025   

  AttStatus Region           Supervisor  Visit_Date   Designation StoreCount  \
0         P   West  Mahalaxmi Ajit Pady  01-11-2025  Merchandiser        3.0   
1         P   West  Mahalaxmi Ajit Pady  02-11-2025  Merchandiser        1.0   
2        WO   West  Mahalaxmi Ajit Pady                                        
3         P   West  Mahalaxmi Ajit Pady  04-11-2025  Merchandiser        1.0   
4         P   West  Mahalaxmi Ajit Pady  05-11-2025  Merchandiser        2.0   

  Store_CheckIn Store_CheckOut    FOS FOS_MoreThan_6_30  
0      11:45:50       20:48:32  09:02               1.0  
1      11:09:08       20:21:49  09:12               1.0  
2                                                        
3      10:58:15       20:30:47  09:32               1.0  
4      11:02:49       20:35:05  09:32               1.0